### Lending Club P2P loans 2018 Q1

The rise of peer-to-peer (P2P) lending in recent years has contributed greatly to democratizing access to financing for previously underserved population groups. What are the characteristics of such borrowers and what can we learn about their credit habits?

Thankfully, Lending Club releases quarterly data on loans issued during a particular period. I will be using the most recent loan data on 2018 Q1 to look at the most recent batch of borrowers. Understandably, due to the recency of the data, repayment information is still incomplete. It would be interesting down the road to look at an older data set with more repayment information or at the declined loans data that Lending Club provides.

In [1]:
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import missingno as msno
from itertools import islice
import csv

In [4]:
all_data = []
with open('LoanStats_2018Q1.csv') as data:
    for row in islice(csv.reader(data), 1, None):
        all_data.append(np.array(row))
        
df = pd.DataFrame(all_data)
df.columns = df.iloc[0]
df = df.drop([0])
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
1,,,3000,3000,3000,36 months,10.41%,97.39,B,B3,...,,,Cash,N,,,,,,
2,,,14100,14100,14100,36 months,23.87%,552.23,E,E2,...,,,Cash,N,,,,,,
3,,,10000,10000,10000,36 months,6.07%,304.54,A,A2,...,,,Cash,N,,,,,,
4,,,40000,40000,40000,60 months,9.43%,838.71,B,B1,...,,,Cash,N,,,,,,
5,,,6300,6300,6300,36 months,9.43%,201.61,B,B1,...,,,Cash,N,,,,,,


In [11]:
df.shape

(107868, 145)

A look at the shape reveals 107,868 loans originated in Q1 of 2018. There are 145 columns with some columns that are completely empty.

Some empty columns such as id and member_id are understandeable they are personally identifiable information. Many of the variables also relate to detailed loan information. For the purposes of this analysis, we focus on a few demographic variables and basic loan information:

1. loan_amnt
2. funded_amnt